In [1]:
import pandas as pd
import numpy as np
import os
import requests
import re

!pip install transformers -q
!pip install wandb -q

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

!pip install SentencePiece -q

import torch.nn.functional as F
from transformers import  T5Tokenizer , T5ForConditionalGeneration
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

%pip install -q wandb
import wandb
wandb.login()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Note: you may need to restart the kernel to use updated packages.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
df = pd.read_csv('/kaggle/input/dataisoil/test.csv')

In [19]:
df = df[['Article','Summary']]
df

,Article,Summary
0,Fear shakes Mexico border city after violence ...,The attacks took place in several neighborhood...
1,Indian-origin boy finds millions of years old ...,"Siddak Singh Jhamat, known as Sid, was using a..."
2,Representative ImageA 38-year-old Indian man h...,A 38-year-old Indian man has been charged with...
3,Residents get tested during their stay at a te...,"China on Apr 21 reported 2,119 locally transmi..."
4,Israeli PM Benjamin Netanyahu flew to Saudi Ar...,Israeli media reported Monday that Prime Minis...
...,...,...
4482,FILE - Tanks storing treated radioactive water...,A massive earthquake and tsunami in 2011 destr...
4483,Villagers clear debris caused by an earthquake...,"Over 680 tourists from France, Thailand, the N..."
4484,"Seif al-Islam, the son and one-time heir appar...","Seif al-Islam, the son and one-time heir appar..."
4485,Thai BoysMost members of the Thai youth footba...,Most members of the Thai youth football team r...


In [20]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, ' ', data)

def preprocess_tokenize(text):
      # for removing punctuation from sentencesc
    text = str(text)
    text = re.sub(r'(\d+)', r'', text)
    
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('\t', ' ')
    text = text.replace('\u200d', '')
    text=re.sub("(__+)", ' ', str(text)).lower()   #remove _ if it occors more than one time consecutively
    text=re.sub("(--+)", ' ', str(text)).lower()   #remove - if it occors more than one time consecutively
    text=re.sub("(~~+)", ' ', str(text)).lower()   #remove ~ if it occors more than one time consecutively
    text=re.sub("(\+\++)", ' ', str(text)).lower()   #remove + if it occors more than one time consecutively
    text=re.sub("(\.\.+)", ' ', str(text)).lower()   #remove . if it occors more than one time consecutively
    text=re.sub(r"[<>()|&©@#ø\[\]\'\",;:?.~*!]", ' ', str(text)).lower() #remove <>()|&©ø"',;?~*!
    text = re.sub(r"[‘’।:]", " ", str(text)) #removing other special characters
    text = remove_emojis(text)
    return text

In [21]:
df['Article'] = [preprocess_tokenize(x) for x in df['Article']]
df

,Article,Summary
0,fear shakes mexico border city after violence ...,The attacks took place in several neighborhood...
1,indian-origin boy finds millions of years old ...,"Siddak Singh Jhamat, known as Sid, was using a..."
2,representative imagea -year-old indian man has...,A 38-year-old Indian man has been charged with...
3,residents get tested during their stay at a te...,"China on Apr 21 reported 2,119 locally transmi..."
4,israeli pm benjamin netanyahu flew to saudi ar...,Israeli media reported Monday that Prime Minis...
...,...,...
4482,file - tanks storing treated radioactive water...,A massive earthquake and tsunami in 2011 destr...
4483,villagers clear debris caused by an earthquake...,"Over 680 tourists from France, Thailand, the N..."
4484,seif al-islam the son and one-time heir appar...,"Seif al-Islam, the son and one-time heir appar..."
4485,thai boysmost members of the thai youth footba...,Most members of the Thai youth football team r...


In [22]:
df.rename(columns = {'Article':'ctext', 'Summary':'text'}, inplace = True)
df.ctext = 'summarize: ' + df.ctext
df

,ctext,text
0,summarize: fear shakes mexico border city afte...,The attacks took place in several neighborhood...
1,summarize: indian-origin boy finds millions of...,"Siddak Singh Jhamat, known as Sid, was using a..."
2,summarize: representative imagea -year-old ind...,A 38-year-old Indian man has been charged with...
3,summarize: residents get tested during their s...,"China on Apr 21 reported 2,119 locally transmi..."
4,summarize: israeli pm benjamin netanyahu flew ...,Israeli media reported Monday that Prime Minis...
...,...,...
4482,summarize: file - tanks storing treated radioa...,A massive earthquake and tsunami in 2011 destr...
4483,summarize: villagers clear debris caused by an...,"Over 680 tourists from France, Thailand, the N..."
4484,summarize: seif al-islam the son and one-time...,"Seif al-Islam, the son and one-time heir appar..."
4485,summarize: thai boysmost members of the thai y...,Most members of the Thai youth football team r...


In [23]:
wandb.init(project="English_t5_pred")

config = wandb.config         
config.TRAIN_BATCH_SIZE = 2  
config.VALID_BATCH_SIZE = 2    
config.TRAIN_EPOCHS = 8       
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 5e-5    
config.SEED = 42               
config.MAX_LEN = 512
config.SUMMARY_LEN = 75 


torch.manual_seed(config.SEED) 
np.random.seed(config.SEED)
torch.backends.cudnn.deterministic = True


tokenizer = T5Tokenizer.from_pretrained("t5-base")

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [12]:
from huggingface_hub import notebook_login
notebook_login()

In [24]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }


In [25]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=75,
                num_beams=4,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [15]:
df

,ctext,text
0,summarize: fear shakes mexico border city afte...,the attacks took place in several neighborhood...
1,summarize: indian-origin boy finds millions of...,siddak singh jhamat known as sid was using a...
2,summarize: representative imagea -year-old ind...,a -year-old indian man has been charged with c...
3,summarize: residents get tested during their s...,china on apr reported locally transmitted c...
4,summarize: israeli pm benjamin netanyahu flew ...,israeli media reported monday that prime minis...
...,...,...
4482,summarize: file - tanks storing treated radioa...,a massive earthquake and tsunami in destroyed...
4483,summarize: villagers clear debris caused by an...,over tourists from france thailand the neth...
4484,summarize: seif al-islam the son and one-time...,seif al-islam the son and one-time heir appar...
4485,summarize: thai boysmost members of the thai y...,most members of the thai youth football team r...


In [26]:
val_dataset = df

In [28]:
val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

# parameters fo dataloaders
train_params = {
    'batch_size': config.TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

# Create  Dataloaders 
val_loader = DataLoader(val_set, **val_params)


model = T5ForConditionalGeneration.from_pretrained("saliq7/T5_English")
model = model.to(device)


optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)
wandb.watch(model, log="all")


#for epoch in range(config.VAL_EPOCHS):
predictions, actuals = validate(1, tokenizer, model, device, val_loader)
final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
final_df.to_csv('predictions_english_t5_preprocessed.csv')
print('Output Files generated for review')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Completed 500
Completed 600
Completed 700
Completed 800
Completed 900
Completed 1000
Completed 1100
Completed 1200
Completed 1300
Completed 1400
Completed 1500
Completed 1700
Completed 1800
Completed 1900
Completed 2000
Completed 2100
Completed 2200
Output Files generated for review


In [29]:
df = pd.read_csv('/kaggle/working/predictions_english_t5_preprocessed.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [30]:
df

,Generated Text,Actual Text
0,gunmen in vehicles killed people including tax...,The attacks took place in several neighborhood...
1,indian-origin boy says he is “really excited” ...,"Siddak Singh Jhamat, known as Sid, was using a..."
2,us attorney s office for new jersey said in a ...,A 38-year-old Indian man has been charged with...
3,shanghai has added new cases in the previous h...,"China on Apr 21 reported 2,119 locally transmi..."
4,israeli media reported monday that prime minis...,Israeli media reported Monday that Prime Minis...
...,...,...
4482,nuclear regulator on wednesday approved plans ...,A massive earthquake and tsunami in 2011 destr...
4483,stranded tourists are now slowly making their ...,"Over 680 tourists from France, Thailand, the N..."
4484,gadhafi's son was captured by the battalion s ...,"Seif al-Islam, the son and one-time heir appar..."
4485,thai officials said the boys will start the pr...,Most members of the Thai youth football team r...


In [31]:
!pip install evaluate -q
!pip install rouge-score -q

In [32]:
import evaluate
rouge = evaluate.load('rouge')
predictions = [i for i in df['Generated Text']]
references = [i for i in df['Actual Text']]
results = rouge.compute(predictions=predictions, references=references)
print(results)

{'rouge1': 0.41039874461132764, 'rouge2': 0.27236401076232064, 'rougeL': 0.35387976568305124, 'rougeLsum': 0.3540848985943519}


In [35]:
!pip install -q rouge

In [36]:
import rouge
from rouge import Rouge

rouge = Rouge()
score = rouge.get_scores(df['Generated Text'], df['Actual Text'], avg = True)
RougeScore = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
RougeScore

,rouge-1,rouge-2,rouge-l
recall,0.321935,0.185400,0.289839
precision,0.313371,0.175907,0.282344
f-measure,0.310759,0.176004,0.280021


In [43]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
score = corpus_bleu(references, predictions)
print(score)

0.6297854632360553


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
